In [1]:
import json
with open('annotated/dataset_annotated-fixed-done.jsonl','r') as f:
    
    ds_calibre = list(map(lambda x : json.loads(x),f.readlines()))

In [2]:
with open('annotated/smaller_dataset_goodreads(ANNOTATED).jsonl','r') as f:
    
    ds_goodreads = list(map(lambda x : json.loads(x),f.readlines()))

In [3]:
with open('annotated/odyssey_annotaded.jsonl','r') as f:
    
    ds_cliff = list(map(lambda x : json.loads(x),f.readlines()))

#### Goodreads dataset needs preproc

In [4]:
for d in ds_goodreads:
    book = d['meta']['book']
    
    temp = d['text'].replace('[[BOOK]]',book)
    d['text'] = temp
    b = temp.find(book)
    e = b + len(book)

    #dict(start=5,end=14,text="Tal Perry",label="Person")
    entities = {
        'start' : b,
        'end' : e,
        'text' : book,
        'label' : "OBRA",
        'tag' : "OBRA"
    }
    if(d['labels']==['negative']):
        d['entities'] = []
    else:
        d['entities'] = [entities]

In [5]:
with open('dataset_goodreads.jsonl','r') as f:
    
    ds_goodreads_not_annotated = list(map(lambda x : json.loads(x),f.readlines()))

In [6]:
books = list(set([d['meta']['book'] for d in ds_goodreads_not_annotated]))

In [7]:
books[:10]

['Untouchable',
 'Love Poems',
 'The Untouchables',
 'Ludwig Wittgenstein',
 'Rebellion',
 'Angels',
 'Hideaway',
 'Moby Dick',
 'Velocity',
 'Moll Flanders']

## change books in a dataset to other random books, so the NER model doesn't memorize a specific book

In [9]:
import numpy as np

np.random.seed(1337)

for d in ds_cliff:
    book_to_change = np.random.choice(books,1)[0]
    book = 'The Odyssey'
    
    if (d['labels']!=[['negative']]):
        temp = d['text'].replace(book,book_to_change)
        d['text'] = temp
        b = temp.find(book_to_change)
        e = b + len(book_to_change)

        #dict(start=5,end=14,text="Tal Perry",label="Person")
        entities = {
            'start' : b,
            'end' : e,
            'text' : book_to_change,
            'label' : "OBRA",
            'tag' : "OBRA"
        }

        d['meta']['book'] = book_to_change
    
    if(d['labels']==[['negative']]):
        d['entities'] = []
    else:
        d['entities'] = [entities]
        
# delete entries without tags
#for i in range(len(ds_cliff)):
    #if(ds_cliff[i]['labels']==[['negative']]):
        #del ds_cliff[i]
        
for d in ds_calibre:
    
    if d['entities']:
        book_to_change = np.random.choice(books,1)[0]
        book = d['entities'][0]['text']
        
        temp = d['text'].replace(book,book_to_change)
        d['text'] = temp
        b = temp.find(book_to_change)
        e = b + len(book_to_change)

        #dict(start=5,end=14,text="Tal Perry",label="Person")
        entities = {
            'start' : b,
            'end' : e,
            'text' : book_to_change,
            'label' : "OBRA",
            'tag' : "OBRA"
        }

        d['meta']['book'] = book_to_change

        d['entities'] = [entities]

In [10]:
ds = []

In [11]:
ds.extend(ds_calibre)
ds.extend(ds_cliff)

In [12]:
import random

random.shuffle(ds)

dataset_train = ds[:-50]
dataset_test = ds[-50:]

In [13]:
#with open('ner_datasets/dataset_annotated_ner_calibre.jsonl','w') as f:
#    for js in ds: 
#        f.write(json.dumps(js) + '\n')
#    f.close()

### convert ENTITIES to Hugginface Format (tokens)

In [14]:
from transformers import RobertaTokenizerFast, RobertaModel
import torch

tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')


/home/lira/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lira/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lira/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lira/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

In [15]:
from typing_extensions import TypedDict
from typing import List,Any
IntList = List[int] # A list of token_ids
IntListList = List[IntList] # A List of List of token_ids, e.g. a Batch

def align_tokens_and_annotations_bilou(tokenized, annotations):
    tokens = tokenized.tokens
    aligned_labels = ["O"] * len(
        tokens
    )  # Make a list to store our labels the same length as our tokens
    for anno in annotations:
        annotation_token_ix_set = (
            set()
        )  # A set that stores the token indices of the annotation
        for char_ix in range(anno["start"], anno["end"]):

            token_ix = tokenized.char_to_token(char_ix)
            if token_ix is not None:
                annotation_token_ix_set.add(token_ix)
        if len(annotation_token_ix_set) == 1:
            # If there is only one token
            token_ix = annotation_token_ix_set.pop()
            prefix = (
                "U"  # This annotation spans one token so is prefixed with U for unique
            )
            aligned_labels[token_ix] = f"{prefix}-{anno['label']}"

        else:

            last_token_in_anno_ix = len(annotation_token_ix_set) - 1
            for num, token_ix in enumerate(sorted(annotation_token_ix_set)):
                if num == 0:
                    prefix = "B"
                elif num == last_token_in_anno_ix:
                    prefix = "L"  # Its the last token
                else:
                    prefix = "I"  # We're inside of a multi token annotation
                aligned_labels[token_ix] = f"{prefix}-{anno['label']}"
    return aligned_labels


import itertools


class LabelSet:
    def __init__(self, labels: List[str]):
        self.labels_to_id = {}
        self.ids_to_label = {}
        self.labels_to_id["O"] = 0
        self.ids_to_label[0] = "O"
        num = 0  # in case there are no labels
        # Writing BILU will give us incremntal ids for the labels
        for _num, (label, s) in enumerate(itertools.product(labels, "BILU")):
            num = _num + 1  # skip 0
            l = f"{s}-{label}"
            self.labels_to_id[l] = num
            self.ids_to_label[num] = l
        # Add the OUTSIDE label - no label for the token

    def get_aligned_label_ids_from_annotations(self, tokenized_text, annotations):
        raw_labels = align_tokens_and_annotations_bilou(tokenized_text, annotations)    
        return list(map(self.labels_to_id.get, raw_labels))
from dataclasses import dataclass
from torch.utils.data import Dataset
from transformers import PreTrainedTokenizerFast

@dataclass
class TrainingExample:
    input_ids: IntList
    attention_masks: IntList
    labels: IntList


class TraingDataset(Dataset):
    def __init__(
        self,
        data: Any,
        label_set: LabelSet,
        tokenizer: PreTrainedTokenizerFast,
        tokens_per_batch=32,
        window_stride=None,
    ):
        self.label_set = label_set
        if window_stride is None:
            self.window_stride = tokens_per_batch
        self.tokenizer = tokenizer
        for example in data:
            # changes tag key to label
            for a in example["entities"]:
                a["label"] = a["tag"]
                pass
        self.texts = []
        self.annotations = []

        for example in data:
            self.texts.append(example["text"])
            self.annotations.append(example["entities"])
        ###TOKENIZE All THE DATA
        tokenized_batch = self.tokenizer(self.texts, add_special_tokens=False)
        ###ALIGN LABELS ONE EXAMPLE AT A TIME
        aligned_labels = []
        for ix in range(len(tokenized_batch.encodings)):
            encoding = tokenized_batch.encodings[ix]
            raw_annotations = self.annotations[ix]            
            aligned = label_set.get_aligned_label_ids_from_annotations(
                encoding, raw_annotations
            )
            aligned_labels.append(aligned)
        ###END OF LABEL ALIGNMENT

        ###MAKE A LIST OF TRAINING EXAMPLES. (This is where we add padding)
        self.training_examples: List[TrainingExample] = []
        empty_label_id = "O"
        for encoding, label in zip(tokenized_batch.encodings, aligned_labels):
            length = len(label)  # How long is this sequence
            for start in range(0, length, self.window_stride):

                end = min(start + tokens_per_batch, length)

                # How much padding do we need ?
                padding_to_add = max(0, tokens_per_batch - end + start)
                self.training_examples.append(
                    TrainingExample(
                        # Record the tokens
                        input_ids=encoding.ids[start:end]  # The ids of the tokens
                        + [self.tokenizer.pad_token_id]
                        * padding_to_add,  # padding if needed
                        labels=(
                            label[start:end]
                            + [-100] * padding_to_add  # padding if needed
                        ),  # -100 is a special token for padding of labels,
                        attention_masks=(
                            encoding.attention_mask[start:end]
                            + [0]
                            * padding_to_add  # 0'd attenetion masks where we added padding
                        ),
                    )
                )

    def __len__(self):
        return len(self.training_examples)

    def __getitem__(self, idx) -> TrainingExample:

        return self.training_examples[idx]

In [17]:
class TraingingBatch:
    def __getitem__(self, item):
        return getattr(self, item)

    def __init__(self, examples: List[TrainingExample]):
        self.input_ids: torch.Tensor
        self.attention_masks: torch.Tensor
        self.labels: torch.Tensor
        input_ids: IntListList = []
        masks: IntListList = []
        labels: IntListList = []
        for ex in examples:
            input_ids.append(ex.input_ids)
            masks.append(ex.attention_masks)
            labels.append(ex.labels)
        self.input_ids = torch.LongTensor(input_ids)
        self.attention_masks = torch.LongTensor(masks)
        self.labels = torch.LongTensor(labels)

In [18]:
label_set = LabelSet(labels=["OBRA"])

ds = TraingDataset(
    data=dataset_train, tokenizer=tokenizer, label_set=label_set, tokens_per_batch=100
)

In [38]:
from transformers.optimization import get_linear_schedule_with_warmup
from transformers import AdamW
from torch.utils.data.dataloader import DataLoader
from transformers import RobertaForTokenClassification



model = RobertaForTokenClassification.from_pretrained(
    "roberta-base", num_labels=len(ds.label_set.ids_to_label.values())
)

optimizer = AdamW(model.parameters(), lr=5e-6)
nb_grad_acc_steps = 4
warmup_steps = 5
num_epochs = 5

dataloader = DataLoader(
    ds,
    collate_fn=TraingingBatch,
    batch_size=3,
    shuffle=True,
)



nb_training_steps = len(dataloader)//nb_grad_acc_steps

scheduler = get_linear_schedule_with_warmup(optimizer,warmup_steps,num_training_steps=nb_training_steps)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able

In [39]:
import numpy as np
from sklearn.metrics import classification_report

def check_accuracy_model(model,dataset_raw):

    outs = []
    for d in dataset_raw:

        
            if len(d['entities'])>0:
                y_true = d['entities'][0]['text']
            else:
                y_true = False

            inputs = tokenizer(d['text'], return_tensors="pt",truncation=True)

            logits = model(**inputs)

            idx = torch.nonzero(torch.argmax(logits[0][0],axis=1))
            if (len(idx)):
                ids_labeled = inputs.input_ids[0][idx]
                tokens = tokenizer.convert_ids_to_tokens(ids_labeled)
                y_pred = tokenizer.convert_tokens_to_string(tokens).strip()
                
            else:
                y_pred = False
            
            
            # a true prediction only counts if the model found the correct book
            if y_pred:
                if y_true:
                    flag = False
                    for s in y_pred.split():
                        if y_true.find(s) != -1:
                            temp = y_true.find(y_pred) != -1
                            if(temp):
                                flag = True
                    y_pred = flag
                    y_true = True
                else:
                    y_pred = False
                    y_true = True
            else:
                if y_true:
                    y_true = True
            outs.append((y_true,y_pred))
            
                
                

    y_true = list(map(lambda x : x[0] , outs))
    y_pred = list(map(lambda x : x[1] , outs))

    return classification_report(y_true, y_pred, target_names=['positive','negative'],output_dict=True)['accuracy']

In [40]:
from tqdm import tqdm





for epoch in range(num_epochs):
    iterator = dataloader.__iter__()

    loop = tqdm(range(int(nb_training_steps)))
    
    for i_train in loop:
        
        loss=0
        optimizer.zero_grad()
        
        for i_acc in range(nb_grad_acc_steps):
            
            batch = next(iterator)


            loss_, logits = model(
                input_ids=batch.input_ids,
                attention_mask=batch.attention_masks,
                labels=batch.labels,
            )
            loss+=loss_

        loss=loss/nb_grad_acc_steps
        loss.backward()
        
        optimizer.step()
        scheduler.step()

        loop.set_description(f'Epoch={epoch}/{num_epochs}')
        loop.set_postfix(loss=loss.item())
        
    gd_val = check_accuracy_model(model,ds_goodreads)
    val_val = check_accuracy_model(model,dataset_test)

    print(f'Epoch val loss Goodreads:{gd_val} Val Set:{val_val}')
    

  0%|          | 0/295 [00:00<?, ?it/s]

Epoch val loss Goodreads:0.7200902934537246 Val Set:0.72


  0%|          | 0/295 [00:00<?, ?it/s]

Epoch val loss Goodreads:0.7200902934537246 Val Set:0.72


Epoch=2/5:   1%|          | 2/295 [00:08<21:39,  4.44s/it, loss=0.0232]


KeyboardInterrupt: 

In [99]:
ds.label_set.ids_to_label.values()

dict_values(['O', 'B-OBRA', 'I-OBRA', 'L-OBRA', 'U-OBRA'])

In [41]:
from utils import check_if_citation_ner

In [42]:
check_if_citation_ner(model,tokenizer,'Reading The Prince and The Republic has changed my life')

''

In [43]:
check_if_citation_ner(model,tokenizer,'The prince reads Marcus Aurelius\' Meditations to relax')

''

In [44]:
check_if_citation_ner(model,tokenizer,"When Odysseus returns to Ithaca in Book 13 of The Odyssey, Athena disguises him as an old beggar")

''

In [45]:
check_if_citation_ner(model,tokenizer,"But these more outré materials combine to form what is finally a thematic second tier. The foreground of Infinite Jest features three basic plot systems. At the center of one is Hal Incandenza, an adolescent tennis star attending Enfield Tennis Academy (ETA), which his family founded,")

''

In [46]:
check_if_citation_ner(model,tokenizer,"the commencement of war a herald might be called upon to recite the causes of the conflict; in effect, to provide the motivation. In Shakespeare's Henry IV")

''

In [47]:
check_if_citation_ner(model,tokenizer,"is impersonal in the Meditations agrees closely with Epictetus. Marcus Aurelius is doubtful about immortality, but says, as a Christian might: 'Since it is possible th")

''

In [48]:
check_if_citation_ner(model,tokenizer,"8 Melville’s Moby-Dick, for instance.")

''

In [147]:
import os
model.save_pretrained(f"fine-tuned-model-ner-better-data-2")

In [150]:
for d in dataset_test:
    
    w = check_if_citation_ner(model,tokenizer,d['text'])
    print(d['entities'],w,)

[] 
[{'start': 18, 'end': 37, 'text': 'Nicomachaean Ethics', 'label': 'OBRA', 'tag': 'OBRA'}] Nicomachaean Ethics
[{'start': 3, 'end': 23, 'text': 'Occidental Mythology', 'label': 'OBRA', 'tag': 'OBRA'}] Occidental Mythology
[] 
[] 
[] 
[{'start': 14, 'end': 32, 'text': 'Nicomachean Ethics', 'label': 'OBRA', 'tag': 'OBRA'}] Nicomachean Ethics
[{'start': 13, 'end': 24, 'text': 'The Odyssey', 'label': 'OBRA', 'tag': 'OBRA'}] The Odyssey
[] 
[] 
[] 
[] 
[{'start': 14, 'end': 33, 'text': 'Nicomachaean Ethics', 'label': 'OBRA', 'tag': 'OBRA'}] Nicomachaean Ethics
[] 
[] 
[] 
[] 
[{'start': 16, 'end': 35, 'text': 'Nicomachaean Ethics', 'label': 'OBRA', 'tag': 'OBRA'}] Nicomachaean Ethics
[{'start': 296, 'end': 302, 'text': 'Ethics', 'label': 'OBRA', 'tag': 'OBRA'}] Ethics
[] 
[] 
[] 
[] 
[] 
[] 
[] 
[{'start': 539, 'end': 559, 'text': 'Occidental Mythology', 'label': 'OBRA', 'tag': 'OBRA'}] Occidental Mythology
[{'start': 95, 'end': 101, 'text': 'Hamlet', 'label': 'OBRA', 'tag': 'OBRA'}] Ham

IndexError: index out of range in self

In [149]:
d

{'id': 22896,
 'text': 'To connect the force which produces the phenomenon of the world, and consequently determines its nature, with the morality of the disposition or character, and thus to establish a moralorder of the world as the foundation of the physical,—this has been since Socrates the problem of philosophy. Theism solved it in a childish manner, which could not satisfy mature humanity. Therefore pantheism opposed itself to it whenever it ventured to do so, and showed that nature bears in itself the power by virtue of which it appears. With this, however, ethics had necessarily to be given up. Spinoza, indeed, attempts here and there to preserve it by means of sophistry, but for the most part gives it up altogether, and, with a boldness which excites astonishment and repugnance, explains the distinction between right and wrong, and in general between good and evil, as merely conventional, thus in itself empty (for example, Eth. iv., prop. 37, schol. 2). After having met with u

In [2]:
from transformers import RobertaTokenizer, RobertaModel,RobertaForTokenClassification


/home/lira/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lira/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lira/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lira/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

In [4]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForTokenClassification.from_pretrained('fine-tuned-model-ner-better-data-2/')

In [5]:
check_if_citation_ner(model,tokenizer,'So Plato reads The Republic and then replies to Thrasymachus and Callicles, and to all Nietzscheans forever: Justice is not mere strength, but harmonious strength—desires and men falling into that order which constitutes intelligence and organization; justice is not the right of the stronger, but the effective harmony of the whole. It is true that the individual who gets out of the place to which his nature and talents adapt him may for a time seize some profit and advantage; but an inescapable [[BOOK]] pursues him—as Anaxagoras spoke of the Furies pursuing any planet that should wander out of its orbit; the terrible baton of the Nature of Things drives the refractory instrument back to its place and its pitch and its natural note."')

NameError: name 'check_if_citation_ner' is not defined

In [79]:
import json

with open('annotated/unused/dataset_annotated_doccano_calibre.jsonl','r') as f:
    
    ds_ = list(map(lambda x : json.loads(x),f.readlines()))
    

In [80]:
"entities": [{"start": 4, "end": 25, "text": "Shakespeare's Sonnets", "label": "OBRA", "tag": "OBRA"}]

SyntaxError: illegal target for annotation (<ipython-input-80-15a66747d392>, line 1)

In [81]:
# convert to non-doccano format
for d in ds_:
    d['entities'] = []
    
    if (d['labels']): 
        for l in d['labels']:
            b =  l[0]
            e = l[1]

            book = d['text'][b:e]

            entities = {
                'start' : b,
                'end' : e,
                'text' : book,
                'label' : "OBRA",
                'tag' : "OBRA"
            }
            d['entities'].append(entities)


In [82]:
with open('annotated/dataset_annotated-fixed-done.jsonl','w') as f:
    for js in ds_: 
        f.write(json.dumps(js) + '\n')
    f.close()

In [8]:

for d in ds_:
    book = d['meta']['book']
    
    temp = d['text'].replace('[[BOOK]]',book)
    d['text'] = temp
    b = temp.find(book)
    e = b + len(book)
    if(d['labels']==['negative']):
        del d['entities']
        d['labels']=[]
    else:
        #dict(start=5,end=14,text="Tal Perry",label="Person")
        entities = {
            'start' : b,
            'end' : e,
            'text' : book,
            'label' : "OBRA",
            'tag' : "OBRA"
        }
        d['labels'] = [[b,e,"OBRA"]]
        #d['entities'] = [entities]
        del d['entities']

In [9]:
with open('annotated/dataset_annotated-to_fix-doccano-format.jsonl','w') as f:
    for js in ds_: 
        f.write(json.dumps(js) + '\n')
    f.close()

In [11]:
import torch

book = "The Republic"

inputs = tokenizer("Read The Republic and Moby-Dick", return_tensors="pt",truncation=True)

logits = model(**inputs)

idx = torch.nonzero(torch.argmax(logits[0][0],axis=1))
if (len(idx)):
    ids_labeled = inputs.input_ids[0][idx]
    tokens = tokenizer.convert_ids_to_tokens(ids_labeled)
    y_pred = tokenizer.convert_tokens_to_string(tokens).strip()

for word in y_pred.split():
    if book.find(word):
        return True
return False

o
o


In [10]:
y_pred.split()
    

['The', 'Republic', 'MobyDick']